# **QSVT approach for Hamiltonian simulation:**


The QSVT is a general technique for applying block encoding of matrix polynomials, via quantum signal processing.  For the usage of QSVt we rely on the [Hamiltonian Simulation](https://github.com/Classiq/classiq-library/blob/main/tutorials/hamiltonian_simulation/hamiltonian_simulation_with_block_encoding/hamiltonian_simulation_with_block_encoding.ipynb) notebook by Classiq. 

#### Defining all the prerequisite inputs (Preprocessing)

In [26]:
## Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
import typing
import itertools
import scipy
import classiq
from classiq import *
from classiq.execution import ExecutionPreferences, ClassiqBackendPreferences, ClassiqSimulatorBackendNames, ExecutionSession
from typing import cast
from qutip import qeye, sigmax, sigmay, sigmaz
from itertools import product
from numpy import kron
from scipy.special import eval_chebyt, jv
from classiq.qmod.symbolic import pi
import pyqsp
from pyqsp.angle_sequence import Polynomial, QuantumSignalProcessingPhases
from ccho_helpers import *


## Defining all Global Variables
PAULI_DICT = {
    "I": np.array([[1, 0], [0, 1]], dtype=np.complex128),
    "Z": np.array([[1, 0], [0, -1]], dtype=np.complex128),
    "X": np.array([[0, 1], [1, 0]], dtype=np.complex128),
    "Y": np.array([[0, -1j], [1j, 0]], dtype=np.complex128),
}

CHAR_TO_STUCT_DICT = {"I": Pauli.I, "X": Pauli.X, "Y": Pauli.Y, "Z": Pauli.Z}

In [27]:
def is_power_of_2(n):
    """Check if a number is a power of 2."""
    return (n & (n - 1) == 0) and n != 0

def initialize_system():
    while True:
        N = int(input("Enter the number of masses (N): "))
        if is_power_of_2(N):
            break
        else:
            print("Error: N must be a power of 2. Please try again.")

    # Initialize the mass matrix (M) and spring constant matrix (K)
    M = np.zeros((N, N))
    K = np.zeros((N, N))

    # Input the values of the masses
    for i in range(N):
        M[i, i] = float(input(f"Enter the mass m_{i+1}: "))

    # Input the spring constants
    for i in range(N):
        for j in range(i, N):
            k = float(input(f"Enter the spring constant k_{i+1}{j+1}: "))
            K[i, j] = k
            K[j, i] = k  # Since K is a symmetric matrix

    # Input the initial position vector (x_0)
    x_0 = np.zeros((N, 1))
    for i in range(N):
        x_0[i, 0] = float(input(f"Enter the initial position x_0_{i+1}: "))

    # Input the initial velocity vector (xdot_0)
    xdot_0 = np.zeros((N, 1))
    for i in range(N):
        xdot_0[i, 0] = float(input(f"Enter the initial velocity xdot_0_{i+1}: "))

    return N, M, K, x_0, xdot_0

### For demonstration we take a system with 2 unit masses, each connected to neighbour with spring constants of 1, also we take initial positions to be [0,1] and initial velocities [0,0]

In [28]:
N, M, K, x_0, xdot_0 = initialize_system() 

# F Matrix
F= create_matrix_F(K, N)

# A Matrix
A = create_matrix_A(M, F)

# Transform coordinates
y = coordinate_transformation(M, x_0, xdot_0)
y_0 = y["y_0"]
ydot_0 = y["ydot_0"]

# B Matrix
B = create_matrix_B(M, K, A, N)

# Stacking B with zeros to get square matrix N^2 x N^2
B_padded = padding_B(B, N)

# Hamiltonian
Ham = create_Hamiltonian(B_padded)

# Initial State
init_state = create_init_state(B_padded, y_0, ydot_0, N)
E0_y = calculate_energy(y_0, ydot_0, M, K)

# Normalize the Initial State
normalization = normalize_init_state(init_state)
normalized_init_state = normalization["normalized_init_state"]
norm = normalization["norm"]

# Print the results

print("Number of masses", N)
print("Mass matrix", M)
print("Spring Constant Matrix", K)
print("Initial Energy: ", E0_y)
print("B matrix: ", B_padded)
print("Hamiltonian formed:", Ham)
print("Initial State:", init_state)

assert np.matmul(B, B.conj().T).all()==A.all()

Error: N must be a power of 2. Please try again.
Number of masses 2
Mass matrix [[1. 0.]
 [0. 1.]]
Spring Constant Matrix [[1. 1.]
 [1. 1.]]
Initial Energy:  1.0
B matrix:  [[ 1.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j]]
Hamiltonian formed: [[-0.-0.j -0.-0.j -0.-0.j -0.-0.j -1.-0.j -1.-0.j -0.-0.j -0.-0.j]
 [-0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j  1.-0.j -1.-0.j -0.-0.j]
 [-0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j]
 [-0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j]
 [-1.+0.j -0.+0.j -0.+0.j -0.+0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j]
 [-1.+0.j  1.+0.j -0.+0.j -0.+0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j]
 [-0.+0.j -1.+0.j -0.+0.j -0.+0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j]
 [-0.+0.j -0.+0.j -0.+0.j -0.+0.j -0.-0.j -0.-0.j -0.-0.j -0.-0.j]]
Initial State: [[ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [-0.-1.j]
 [ 0.+1.j]
 [ 0.+0.j]]


In [29]:
# Create the Pauli Matrix Decomposition of the Hamiltonian
pauli_list = lcu_naive(Ham)

# Transform Pauli Matrix Decomposition to Classiq compatible
classiq_pauli_list = pauli_list_to_hamiltonian(pauli_list)

print(pauli_list)

100%|██████████| 64/64 [00:00<00:00, 25592.09it/s]

[('XIZ', (-0.5+0j)), ('XIX', (-0.25+0j)), ('XZZ', (-0.5+0j)), ('XZX', (-0.25+0j)), ('XXX', (-0.25+0j)), ('XYY', (-0.25+0j)), ('YIY', (0.25+0j)), ('YZY', (0.25+0j)), ('YXY', (-0.25+0j)), ('YYX', (0.25+0j))]


## QSVT methodology for simulating H

We start with block encoding our problem Hamiltonian using linear combination of unitaries. The QSVT technique then gives a block-encoding for a polynomial of the Hamiltonian (here $f(H)= e^{iHt}= cos(Ht)+isin(Ht)$) with a well defined-parity. Note that we can approximate the sines and cosines using polynomials through Jacobi-Anger Expansion.


Thus, we have to apply two QSVT blocks, one for approximating the $\cos(Ht)$ function and one for the $\sin(Ht)$ function, and then finally construct an LCU to get the block-encoding the sum:$e^{iHt} = \cos(Ht)+i\sin(Ht)$. 



#### Main Principle:

The main goal of QSVT is to find the block-encoding of the polynomial of $H$ as
$$
P(H)= e^{i \phi_0 \sigma_z} U(H) e^{i \phi_1 \sigma_z} U(H) e^{i \phi_2 \sigma_z} \cdot \ldots \cdot U(H) e^{i \phi_k \sigma_z}= \begin{pmatrix}
\cos(Ht) & * \\
* & *
\end{pmatrix} + i \begin{pmatrix}
\sin(Ht) & * \\
* & *
\end{pmatrix}
$$ 

where $\phi_i$ are called QSVT angles and $U(H)$ is the block-encoding of given $H$ found using LCU. 

## Finding block encoding for H

In [30]:
# getting the normalized coefficents representing probabilities for the prepare block of LCU

def get_normalized_lcu_coef(lcu_coef):
    # Calculate the normalization factor as the sum of moduli of the complex numbers
    normalization_factor = sum(np.abs(c) for c in lcu_coef)
 
    # Calculate the prepare_prob list as specified
    prepare_prob = [(np.abs(c) / normalization_factor) for c in lcu_coef]
    
    # Print the prepare_prob list
    print("The prepared probabilities:", prepare_prob)
    
    # Calculate the size of the block encoding
    coef_size = int(np.ceil(np.log2(len(prepare_prob))))
    
    # Pad prepare_prob with zeros to make its length a power of 2
    prepare_prob += [0] * (2**coef_size - len(prepare_prob))

    if sum(prepare_prob)>1:
        k= sum(prepare_prob)-1 
        prepare_prob[0]-=k
    
    if sum(prepare_prob)<1:
        k= 1- sum(prepare_prob) 
        prepare_prob[0]+=k
    
    # Print the details
    print("The size of the block encoding:", coef_size)
    print("The normalized coefficients:", prepare_prob)
    print("The normalization factor:", normalization_factor)
    
    return normalization_factor, coef_size, prepare_prob


@qfunc
def apply_pauli_term(pauli_string: PauliTerm, x: QArray[QBit]):
    repeat(
        count=x.len,
        iteration=lambda index: switch(
            pauli_string.pauli[index],
            [
                lambda: IDENTITY(x[pauli_string.pauli.len - index - 1]),
                lambda: X(x[pauli_string.pauli.len - index - 1]),
                lambda: Y(x[pauli_string.pauli.len - index - 1]),
                lambda: Z(x[pauli_string.pauli.len - index - 1]),
            ],
        ),
    )
    
@qfunc
def lcu_paulis(
    pauli_terms_list: CArray[PauliTerm],
    probs: CArray[CReal],
    block: QNum,
    data: QArray[QBit],
):
    within_apply(
        lambda: inplace_prepare_state(probs, 0.0, block),
        lambda: repeat(
            count=pauli_terms_list.len,
            iteration=lambda i: control(
                block == i, lambda: apply_pauli_term(pauli_terms_list[i], data)
            ),       
            ),
    )
      

In [31]:
lcu_pauli_coef = [p.coefficient for p in classiq_pauli_list]
normalization_ham, lcu_size_ham, prepare_probs_ham = get_normalized_lcu_coef(
    lcu_pauli_coef
)

The prepared probabilities: [0.16666666666666666, 0.08333333333333333, 0.16666666666666666, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333]
The size of the block encoding: 4
The normalized coefficients: [0.16666666666666666, 0.08333333333333333, 0.16666666666666666, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0, 0, 0, 0, 0, 0]
The normalization factor: 3.0


In [32]:
def get_cheb_coef(epsilon, t):
    poly_degree = int(
        np.ceil(
            t
            + np.log(epsilon ** (-1)) / np.log(np.exp(1) + np.log(epsilon ** (-1)) / t)
        )
    )
    cos_coef = [jv(0, t)] + [
        2 * jv(2 * k, t) * (-1) ** k for k in range(1, poly_degree // 2 + 1)
    ]
    sin_coef = [
        -2 * jv(2 * k - 1, t) * (-1) ** k for k in range(1, poly_degree // 2 + 1)
    ]
    return cos_coef, sin_coef

# change W(x) to R(x), as the phases are in the W(x) conventions see Eq.()
def convert_phases_to_Rx_convention(ang_seq):

    phases = np.array(ang_seq)
    phases[1:-1] = phases[1:-1] - np.pi / 2
    phases[0] = phases[0] - np.pi / 4
    phases[-1] = phases[-1] + (2 * (len(phases) - 1) - 1) * np.pi / 4

    # verify conventions. minus is due to exp(-i*phi*z) in qsvt in comparison to qsp
    phases = -2 * phases

    return phases.tolist()



In [46]:

    EPS = 0.1
    EVOLUTION_TIME = 0.5
    # since our hamiltonian from 
    normalized_time = normalization_ham * EVOLUTION_TIME
    cos_coef, sin_coef = get_cheb_coef(EPS, normalized_time)
    combined_sin_cos_coef = []
    for k in range(len(cos_coef) - 1):
        combined_sin_cos_coef.append(cos_coef[k])
        combined_sin_cos_coef.append(sin_coef[k])
    combined_sin_cos_coef.append(cos_coef[-1])
    if len(sin_coef) == len(cos_coef):
        combined_sin_cos_coef.append(sin_coef[-1])
    signs_cheb_coef = np.sign(combined_sin_cos_coef).tolist()
    generalized_signs = [
        (1 - signs_cheb_coef[s]) + (s) % 2 for s in range(len(signs_cheb_coef))
    ]
    positive_cheb_lcu_coef = np.abs(combined_sin_cos_coef)
    normalization_exp, lcu_size_exp, prepare_probs_exp = get_normalized_lcu_coef(
        positive_cheb_lcu_coef
    )



    pg_cos = pyqsp.poly.PolyCosineTX()
    pcoefs_cos, scale_cos = pg_cos.generate(
        epsilon=EPS, tau=normalized_time, return_scale=True)
    poly_cos = Polynomial(pcoefs_cos)
    pg_sin = pyqsp.poly.PolySineTX()
    pcoefs_sin, scale_sin = pg_sin.generate(
        epsilon=EPS, tau=normalized_time, return_scale=True)
    poly_sin = Polynomial(pcoefs_sin)
    ang_seq_cos = QuantumSignalProcessingPhases(
        poly_cos,
        signal_operator="Wx",
        method="laurent",
        measurement="x",
        tolerance=0.000001)  # relaxing the tolerance to get convergence
    ang_seq_sin = QuantumSignalProcessingPhases(
        poly_sin,
        signal_operator="Wx",
        method="laurent",
        measurement="x",
        tolerance=0.000001) # relaxing the tolerance to get convergence

    
    phases_cos = convert_phases_to_Rx_convention(ang_seq_cos)
    phases_sin = convert_phases_to_Rx_convention(ang_seq_sin)

The prepared probabilities: [0.22876613577681035, 0.49874981745518693, 0.20746748505329535, 0.05449680935360053, 0.010519752361106768]
The size of the block encoding: 3
The normalized coefficients: [0.22876613577681035, 0.49874981745518693, 0.20746748505329535, 0.05449680935360053, 0.010519752361106768, 0, 0, 0]
The normalization factor: 2.2373401989675137
3.620632980767203
R=1
[PolyCosineTX] rescaling by 0.5.
3.620632980767203
R=1
[PolySineTX] rescaling by 0.5.


In [47]:
@qfunc
def identify_block(num_data_qubits: CInt, qvar: QArray[QBit], qbit: QBit):
    block_qubits = QNum("block_qubits", qvar.len - num_data_qubits, False, 0)
    data = QArray("data", length=num_data_qubits)

    bind(qvar, [block_qubits, data])
    qbit ^= block_qubits == 0
    bind([block_qubits, data], qvar)


# defining a qsvt for the specific example
@qfunc
def my_qsvt(phases: CArray[CReal], qsvt_aux: QBit, block_ham: QArray, data: QArray):
    combined_vars = QArray("combined_vars")
    bind([block_ham, data], combined_vars)

    qsvt(
        phase_seq=phases,
        proj_cnot_1=lambda qvar, qbit: identify_block(data.len, qvar, qbit),
        proj_cnot_2=lambda qvar, qbit: identify_block(data.len, qvar, qbit),
        u=lambda x: lcu_paulis(
            classiq_pauli_list,
            prepare_probs_ham,
            x[0 : block_ham.len],
            x[block_ham.len : x.len],
        ),
        qvar=combined_vars,
        aux=qsvt_aux,
    )

    bind(combined_vars, [block_ham, data])

In [35]:

@qfunc
def init_state_phase(state: QNum):
    """
        Definition:
            Imply -pi/2 phase to the last half (msb qubit) of the QNum state in order to add -i factor
        Args:
            state (QNum): Initial state with bare amplitudes
        Outputs:
            state (QNum): Initial state with phase
    """

    state_in_qubit = QArray("state_in_qubit")
    msb = QArray("msb", QBit)
    
    size = np.log2(init_state.size)
    allocate(size, msb)

    bind(state, state_in_qubit)
    repeat(state_in_qubit.len, lambda i: CX(state_in_qubit[i], msb[i]))
    control(msb[size-1], lambda: PHASE(pi/2, state_in_qubit[size-1]))
    bind(state_in_qubit, state)


@qfunc
def main(
    qsvt_aux: Output[QBit],
    block_ham: Output[QNum],
    state: Output[QNum],
    block_exp: Output[QBit]
):
    allocate(1, qsvt_aux)
    allocate(1, block_exp)
    allocate(lcu_size_ham, block_ham)
    prepare_amplitudes(amplitudes=list(normalized_init_state), out=state, bound=0.01)
    init_state_phase(state)
    within_apply(
        lambda: H(block_exp),
        lambda: (
            control(
                block_exp == 0,  # cosine
                lambda: my_qsvt(phases_cos, qsvt_aux, block_ham, state),
            ),
            control(
                block_exp == 1,  # sine
                lambda: (
                    U(0, 0, 0, np.pi / 2, qsvt_aux),  # for the i factor
                    my_qsvt(phases_sin, qsvt_aux, block_ham, state),
                ),
            ),
        ),
    )

Execution_Prefs = ExecutionPreferences(
    num_shots=1,
    backend_preferences=ClassiqBackendPreferences(
        backend_name=ClassiqSimulatorBackendNames.SIMULATOR_STATEVECTOR
    ),
)

qmod = create_model(main)

In [48]:
backend_preferences = ClassiqBackendPreferences(backend_name="simulator_statevector")
model_pref = set_execution_preferences(qmod, ExecutionPreferences(num_shots=1, backend_preferences=backend_preferences))
qprog = synthesize(model_pref)
show(qprog)
write_qmod(qmod, "t=0.5QSVT", decimal_precision=16)

Opening: https://platform.classiq.io/circuit/125fcc1a-33d9-4b68-94be-bf95ca54e6fc?version=0.43.3


In [49]:
    results = execute(qprog).result()
    
    state_result = get_projected_state_vector(
    results, "state", {"block_ham": 0.0, "qsvt_aux": 0.0, "block_exp": 0.0})

    expected_state = (
    1 / normalization_exp * scipy.linalg.expm(1j * Ham * EVOLUTION_TIME) @ init_state
)
    relative_phase = np.angle(expected_state[0] / state_result[0])
    state_result = state_result * np.exp(
    1j * relative_phase)

    # Normalize the final state
    normalized_final_state = normalize_final_state2(state_result)

    # Simplify the final state by neglecting small terms
    simplified_final_state = simplify_final_state(normalized_final_state)

    # Correct the normalization factor that comes from the normalization of initial state and transform the row vector to column vector
    final_state = norm * simplified_final_state[..., None]


    # Get the final position and velocity vectors
    final_results = post_process_final_state(final_state, B_padded, N, y_0)
    y_final = final_results["y_final"]
    ydot_final = final_results["ydot_final"]

    # transform to original coordinates

    x_final_results = back_coordinate_transformation(M, y_final, ydot_final)
    x_final = x_final_results["x_final"]
    xdot_final = x_final_results["xdot_final"]

In [50]:
import scipy.integrate

def equations_of_motion(t, init, N, M, K):
    """
    Definition:
        EOM for classical coupled oscillator system with N masses
    Args:
        t (array): Time array
        init (list): Initial Conditions
        N (int): Number of masses
        M (array): Mass matrix (diagonal)
        K (array): Spring constant matrix
    Output:
        (list): List of final equations
    """
    x = init[:N]
    v = init[N:]
    dxdt = v
    dvdt = np.zeros(N)
    
    for i in range(N):
        if i == 0:
            dvdt[i] = (-K[i, i] * x[i] + K[i, i+1] * (x[i+1] - x[i])) / M[i, i]
        elif i == N-1:
            dvdt[i] = (-K[i, i] * x[i] + K[i, i-1] * (x[i-1] - x[i])) / M[i, i]
        else:
            dvdt[i] = (-K[i, i] * x[i] + K[i, i-1] * (x[i-1] - x[i]) + K[i, i+1] * (x[i+1] - x[i])) / M[i, i]

    return np.concatenate((dxdt, dvdt))

# Initial conditions: [x1(0), x2(0), ..., xN(0), v1(0), v2(0), ..., vN(0)]
initial_conditions = np.concatenate((x_0, xdot_0))

# Time span for the simulation
t_span = (0, 0.5)
t_eval = np.linspace(t_span[0], t_span[1], 100)

# Solve the system of differential equations
solution = scipy.integrate.solve_ivp(equations_of_motion, t_span, initial_conditions.flatten(), t_eval=t_eval, args=(N, M, K))

# Extract the results
t = solution.t
x = solution.y[:N, :]
v = solution.y[N:, :]


# Get the results numerically at the final time step
x_classic = np.array([x[0][-1], x[1][-1]])
xdot_classic = np.array([v[0][-1], v[1][-1]])

print("Position Vectors at t=0.5 classically")
print(x_classic)
print("Velocity Vectors at t=0.5 classically")
print(xdot_classic)

# Compare two solutions
comparision = calculate_error(x_final, xdot_final, x_classic, xdot_classic, N)
amplitude_error = comparision["amplitude_error"]
sign_error = comparision["sign_error"]

print("Percentatage error in positions and velocities", amplitude_error)
print(sign_error)

Position Vectors at t=0.5 classically
[0.11486827 0.76271381]
Velocity Vectors at t=0.5 classically
[ 0.41999214 -0.89941772]
Percentatage error in positions and velocities {'Position of Mass 1': 39.75997311606884, 'Velocity of Mass 1': 93.50896271983244, 'Position of Mass 2': 7.231454798474621, 'Velocity of Mass 2': 24.433569418523714}
{'Position of Mass 1': '-', 'Velocity of Mass 1': '+', 'Position of Mass 2': '+', 'Velocity of Mass 2': '+'}
